Give the average sentiment PER CANTON or per city (more precise is imposible)

In [1]:
import numpy as np
import pandas as pd

In [39]:
twitter=pd.read_json("harvest3r_twitter_data_12-01_0.json");

# File Merger (on the server)!

In [ ]:
import os, os.path
files=[name for name in os.listdir('.') if os.path.isfile(name)];
AllTwitter=[];
AllInstagram=[];
AllNews=[];
a=0;
#Add Loop "for" to go in different directory (january to october)
for i in ['january','february','march','april','mai','june','july','august','september','october']:
    files=[name for name in os.listdir('./'+i+"/") if os.path.isfile(name)]; #Change the path accordingly
    for i in files:
        if files[a].find('twitter')>0:
            try:
                twitter=pd.read_json(i);
                AllTwitter.append(twitter);
            except ValueError:
                print ("JSON object issue");
        elif files[a].find('instagram')>0:
            try:
                instagram=pd.read_json(i);
                AllInstagram.append(instagram);
            except ValueError:
                print ("JSON object issue");
        elif files[a].find('news')>0:
            try:
                news=pd.read_json(i);
                AllNews.append(news);
            except ValueError:
                print ("JSON object issue");
        a=a+1;

#To transform into DataFrame
AllTwitter=pd.concat(AllTwitter, ignore_index=True); #To transform into DataFrame
AllInstagram=pd.concat(AllInstagram, ignore_index=True);
AllNews=pd.concat(AllNews, ignore_index=True)

# File merger (locally)!

In [64]:
import os, os.path
files=[name for name in os.listdir('.') if os.path.isfile(name)];
AllTwitter=[];
AllInstagram=[];
AllNews=[];
a=0;

files=[name for name in os.listdir('.') if os.path.isfile(name)]; 
for i in files:
    if files[a].find('twitter')>0:
        try:
            twitter=pd.read_json(i);
            AllTwitter.append(twitter);
        except ValueError:
            print ("JSON object issue");
    elif files[a].find('instagram')>0:
        try:
            instagram=pd.read_json(i);
            AllInstagram.append(instagram);
        except ValueError:
            print ("JSON object issue");
    elif files[a].find('news')>0:
        try:
            news=pd.read_json(i);
            AllNews.append(news);
        except ValueError:
            print ("JSON object issue");
    a=a+1;

#To transform into DataFrame
AllTwitter=pd.concat(AllTwitter, ignore_index=True); #To transform into DataFrame
AllInstagram=pd.concat(AllInstagram, ignore_index=True);
AllNews=pd.concat(AllNews, ignore_index=True)

JSON object issue


# Code to separate row with sentiment to row without + Write in positive.txt and negative.txt + delete stop_words (useless ones)


# I cannot concatenate, don't know why !

In [25]:
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from langdetect import detect
from nltk.stem import PorterStemmer

def deleteContent(pfile):
    pfile.seek(0)
    pfile.truncate()

additional_stopwords=['.',',','/','(',')','[',']',':','@','\'','?','!','\"',';','&']
TwitterWithPos=[]
TwitterWithNeg=[]
TwitterWithout=[]
fPos = open('positive.txt', 'r+')
fNeg = open('negative.txt', 'r+')
fNeu = open('neutral.txt', 'r+')
#English
#fPosEN = open('positive.txt', 'r+')
#fNegEN = open('negative.txt', 'r+')
#fNeuEN = open('neutral.txt', 'r+')
#French
#fPosFR = open('positive.txt', 'r+')
#fNegFR = open('negative.txt', 'r+')
#fNeuFR = open('neutral.txt', 'r+')
#German
#fPosDE = open('positive.txt', 'r+')
#fNegDE = open('negative.txt', 'r+')
#fNeuDE = open('neutral.txt', 'r+')
#Italian
#fPosIT = open('positive.txt', 'r+')
#fNegIT = open('negative.txt', 'r+')
#fNeuIT = open('neutral.txt', 'r+')

deleteContent(fPos)
deleteContent(fNeg)
deleteContent(fNeu)
twitter['language'] = pd.Series(np.zeros(len(twitter)));

for i in range(0,400): #TO CHANGE !!! len(twitter)
    sentence=twitter.iloc[i]["_source"].get('main');
    language='None';
    try:
        language=detect(sentence); 
    except:
        e=1;
    twitter.ix[i,'language']=language;
    #FIND the language of the current Tweet
    if language=='en' or language=='de' or language=='it' or language=='fr': 
        langDict = {'en': 'english', 'de': 'german', 'it':'italian', 'fr':'french'} 
        stop_words = set(stopwords.words(langDict[language])) 
        word_tokens = word_tokenize(sentence)
        filtered_sentence = []
        for w in word_tokens:
            if w not in stop_words and w not in additional_stopwords and len(w)>=3:
                filtered_sentence.append(w);
    #FIND Tweets without sentiment and place it into a text file per language
    if pd.isnull(twitter._source[i].get('sentiment')):
        TwitterWithout.append(twitter.iloc[[i]])
        try:
            filtered_sentence=' '.join(filtered_sentence)
        except ValueError:
            print('One-word Tweet')  
        fNeu.write(str(filtered_sentence)+"\n");
    #FIND Tweets with sentiment and place it into a text file per language
    else:
        #FIND Tweet with POSITIVE sentiment
        if twitter._source[i].get('sentiment')=='POSITIVE':
            TwitterWithPos.append(twitter.iloc[[i]])
            try:
                filtered_sentence=' '.join(filtered_sentence)
            except ValueError:
                print('One-word Tweet') 
            fPos.write(str(filtered_sentence)+"\n");
        #FIND Tweet with NEGATIVE sentiment
        elif twitter._source[i].get('sentiment')=='NEGATIVE':
            TwitterWithNeg.append(twitter.iloc[[i]])
            try:
                filtered_sentence=' '.join(filtered_sentence)
            except ValueError:
                print('One-word Tweet') 
            fNeg.write(str(filtered_sentence)+"\n");

#To transform into DataFrame       
TwitterWithPos=pd.concat(TwitterWithPos, ignore_index=True)
TwitterWithNeg=pd.concat(TwitterWithNeg, ignore_index=True)
TwitterWithout=pd.concat(TwitterWithout, ignore_index=True)

fPos.close()
fNeg.close()
fNeu.close()

# Last work to classify

Last problem still occuring:

1) I have no clue how to get all the word features in "extract_features" function. Problem linked to the fact we're using: nltk.classify.apply_features(extract_features, all_filtered_sentence_fr_train) 
    
    nltk.classify.apply_features(function, toks)
    
    [function(tok) for tok in toks]
    
2) Problem to compute the accuracy. It worked at some point but could'nt fix it back with all the changes I added.

Stemming Words would be nice feat to add, but need to get one for each language...

In [3]:
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize

example_words = ["python","pythoner","pythoning","pythoned","pythonly"]

ps = PorterStemmer()
for w in example_words:
    print(ps.stem(w))

python
python
python
python
pythonli


In [94]:
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from langdetect import detect
from nltk.stem import PorterStemmer
import time
import string
start_time = time.time()

### CREATION STOP WORDS !!!   
with open('stopwords/de') as f:
    mylist = f.read().splitlines()
stopwords_de = []
for i in mylist:
    stopwords_de.append(i)
    
with open('stopwords/en') as f:
    mylist = f.read().splitlines()
stopwords_en = []
for i in mylist:
    stopwords_en.append(i)
    
with open('stopwords/it') as f:
    mylist = f.read().splitlines()
stopwords_it = []
for i in mylist:
    stopwords_it.append(i)
    
with open('stopwords/fr') as f:
    mylist = f.read().splitlines()
stopwords_fr = []
for i in mylist:
    stopwords_fr.append(i)
### FIN CREATION STOPWORDS
    
def filtering(stop_words, index):
    sentence=twitterAllWith.iloc[index]["_source"].get('main');
    word_tokens = word_tokenize(sentence)
    words_filtered=[x.lower() for x in word_tokens if x not in stop_words if x not in string.punctuation if len(x)>=3 ]
    #COULD BE ADDED IF NEEDED : if x.find('youtu.be')==-1 if x.find('twitter.com')==-1 if x.find('www')==-1 if x.find('.com')==-1
    return words_filtered

def get_words_in_tweets(tweets):
    all_words = []
    for (words, sentiment) in tweets:
        all_words.extend(words)
    return all_words

def get_word_features(wordlist):
    wordlist = nltk.FreqDist(wordlist)
    word_features = wordlist.keys()
    return word_features

def extract_features(document):
    document_words = set(document)
    features = {}
    for word in get_word_features(document):
        features['contains(%s)' % word] = (word in document_words)
    return features

"""
TwitterWithPos=[]
TwitterWithNeg=[]
TwitterWithNeu=[]
TwitterWithout=[]
twitterAllWith=[]
all_filtered_sentence_en=[]
all_filtered_sentence_de=[]
all_filtered_sentence_fr=[]
all_filtered_sentence_it=[]


#DISSOCIATE ROW WITH/WITHOUT SENTIMENT     
#FIND Tweets without sentiment and place it into a text file per language

for i in range(0,len(twitter)):
    if pd.isnull(twitter._source[i].get('sentiment')):
        TwitterWithout.append(twitter.iloc[[i]])
    else:
        #FIND Tweet with POSITIVE sentiment
        if twitter._source[i].get('sentiment')=='POSITIVE':
            TwitterWithPos.append(twitter.iloc[[i]]) 
        #FIND Tweet with NEGATIVE sentiment
        elif twitter._source[i].get('sentiment')=='NEGATIVE':
            TwitterWithNeg.append(twitter.iloc[[i]])
        #FIND Tweet with NEUTRAL sentiment
        elif twitter._source[i].get('sentiment')=='NEUTRAL':
            TwitterWithNeu.append(twitter.iloc[[i]])

#To transform into DataFrame       
TwitterWithPos=pd.concat(TwitterWithPos, ignore_index=True)
TwitterWithNeg=pd.concat(TwitterWithNeg, ignore_index=True)
TwitterWithout=pd.concat(TwitterWithout, ignore_index=True)

frames = [TwitterWithPos, TwitterWithNeg,TwitterWithNeu]
twitterAllWith = pd.concat(frames)
twitterAllWith = twitterAllWith.sample(frac=1).reset_index(drop=True) #Shuffle DF
"""  
TwitterWith=twitter
for i in range(0,len(twitter)):
    if pd.isnull(twitter._source[i].get('sentiment')):
        TwitterWith.drop(TwitterWith.index[[i]])
        
TwitterWith = TwitterWith.sample(frac=1).reset_index(drop=True)

twitter['language'] = pd.Series(np.zeros(len(twitter)));

for i in range(0,len(twitterAllWith)): #TO CHANGE !!! len(twitterAllWith)

    #ADD COLUMN TO ADD THE LANGUAGE
    sentence=twitterAllWith.iloc[i]["_source"].get('main')
    language='None';
    try:
        language=detect(sentence)
    except:
        e=1;
        
    twitterAllWith.ix[i,'language']=language

    if language=='en':
        words_filtered=filtering(stopwords_en,i)
        all_filtered_sentence_en.append((words_filtered,twitterAllWith._source[i].get('sentiment'))) 
    elif language=='fr':
        words_filtered=filtering(stopwords_fr,i)
        all_filtered_sentence_fr.append((words_filtered,twitterAllWith._source[i].get('sentiment')))
    elif language=='de':
        words_filtered=filtering(stopwords_de,i)
        all_filtered_sentence_de.append((words_filtered,twitterAllWith._source[i].get('sentiment')))
    elif language=='it':
        words_filtered=filtering(stopwords_it,i)
        all_filtered_sentence_it.append((words_filtered,twitterAllWith._source[i].get('sentiment')))                        


#CLASSIFYING
#word_features_en = get_word_features(get_words_in_tweets(all_filtered_sentence_en))
all_filtered_sentence_en_train=all_filtered_sentence_en[:round(0.8*len(all_filtered_sentence_en))]
#for i in range(0,len(all_filtered_sentence_en_train)):
#    training_set_en.append(extract_features(all_filtered_sentence_en_train[i],word_features_en))
training_set_en = nltk.classify.apply_features(extract_features, all_filtered_sentence_en_train)
classifier_en = nltk.NaiveBayesClassifier.train(training_set_en)

#word_features_fr = get_word_features(get_words_in_tweets(all_filtered_sentence_fr))
all_filtered_sentence_fr_train=all_filtered_sentence_fr[:round(0.8*len(all_filtered_sentence_fr))]
training_set_fr = nltk.classify.apply_features(extract_features, all_filtered_sentence_fr_train)
classifier_fr = nltk.NaiveBayesClassifier.train(training_set_fr)

#word_features_de = get_word_features(get_words_in_tweets(all_filtered_sentence_de))
all_filtered_sentence_de_train=all_filtered_sentence_de[:round(0.8*len(all_filtered_sentence_de))]
training_set_de = nltk.classify.apply_features(extract_features, all_filtered_sentence_de_train)
classifier_de = nltk.NaiveBayesClassifier.train(training_set_de)

#word_features_it = get_word_features(get_words_in_tweets(all_filtered_sentence_it))
all_filtered_sentence_it_train=all_filtered_sentence_it[:round(0.8*len(all_filtered_sentence_it))]
training_set_it = nltk.classify.apply_features(extract_features, all_filtered_sentence_it_train)
classifier_it = nltk.NaiveBayesClassifier.train(training_set_it)


#PRINT RESULTS
all_filtered_sentence_en_test=all_filtered_sentence_en[round(0.8*len(all_filtered_sentence_en)):]
testing_set_en = nltk.classify.apply_features(extract_features, all_filtered_sentence_en_test)
print(nltk.classify.accuracy(classifier_en, testing_set_en)*100)

all_filtered_sentence_fr_test=all_filtered_sentence_fr[round(0.8*len(all_filtered_sentence_fr)):]
testing_set_fr = nltk.classify.apply_features(extract_features, all_filtered_sentence_fr_test)
print(nltk.classify.accuracy(classifier_fr, testing_set_fr)*100)

all_filtered_sentence_de_test=all_filtered_sentence_de[round(0.8*len(all_filtered_sentence_de)):]
testing_set_de = nltk.classify.apply_features(extract_features, all_filtered_sentence_de_test)
print(nltk.classify.accuracy(classifier_de, testing_set_de)*100)

all_filtered_sentence_it_test=all_filtered_sentence_it[round(0.8*len(all_filtered_sentence_it)):]
testing_set_it = nltk.classify.apply_features(extract_features, all_filtered_sentence_it_test)
print(nltk.classify.accuracy(classifier_it, testing_set_it)*100)

####

print("--- %s seconds ---" % (time.time() - start_time))


ValueError: A ELE probability distribution must have at least one bin.

In [ ]:
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from langdetect import detect
from nltk.stem import PorterStemmer
import time
import string
start_time = time.time()

### CREATION STOP WORDS !!!   
with open('stopwords/de') as f:
    mylist = f.read().splitlines()
stopwords_de = []
for i in mylist:
    stopwords_de.append(i)
    
with open('stopwords/en') as f:
    mylist = f.read().splitlines()
stopwords_en = []
for i in mylist:
    stopwords_en.append(i)
    
with open('stopwords/it') as f:
    mylist = f.read().splitlines()
stopwords_it = []
for i in mylist:
    stopwords_it.append(i)
    
with open('stopwords/fr') as f:
    mylist = f.read().splitlines()
stopwords_fr = []
for i in mylist:
    stopwords_fr.append(i)
### FIN CREATION STOPWORDS
    
def filtering(stop_words, index):
    sentence=twitterAllWith.iloc[index]["_source"].get('main');
    word_tokens = word_tokenize(sentence)
    words_filtered=[x.lower() for x in word_tokens if x not in stop_words if x not in string.punctuation if len(x)>=3 ]
    #COULD BE ADDED IF NEEDED : if x.find('youtu.be')==-1 if x.find('twitter.com')==-1 if x.find('www')==-1 if x.find('.com')==-1
    return words_filtered

def get_words_in_tweets(tweets):
    all_words = []
    for (words, sentiment) in tweets:
        all_words.extend(words)
    return all_words

def get_word_features(wordlist):
    wordlist = nltk.FreqDist(wordlist)
    word_features = wordlist.keys()
    return word_features

def extract_features(document):
    document_words = set(document)
    features = {}
    for word in get_word_features(document):
        features['contains(%s)' % word] = (word in document_words)
    return features

"""
TwitterWithPos=[]
TwitterWithNeg=[]
TwitterWithNeu=[]
TwitterWithout=[]
twitterAllWith=[]
all_filtered_sentence_en=[]
all_filtered_sentence_de=[]
all_filtered_sentence_fr=[]
all_filtered_sentence_it=[]


#DISSOCIATE ROW WITH/WITHOUT SENTIMENT     
#FIND Tweets without sentiment and place it into a text file per language

for i in range(0,len(twitter)):
    if pd.isnull(twitter._source[i].get('sentiment')):
        TwitterWithout.append(twitter.iloc[[i]])
    else:
        #FIND Tweet with POSITIVE sentiment
        if twitter._source[i].get('sentiment')=='POSITIVE':
            TwitterWithPos.append(twitter.iloc[[i]]) 
        #FIND Tweet with NEGATIVE sentiment
        elif twitter._source[i].get('sentiment')=='NEGATIVE':
            TwitterWithNeg.append(twitter.iloc[[i]])
        #FIND Tweet with NEUTRAL sentiment
        elif twitter._source[i].get('sentiment')=='NEUTRAL':
            TwitterWithNeu.append(twitter.iloc[[i]])

#To transform into DataFrame       
TwitterWithPos=pd.concat(TwitterWithPos, ignore_index=True)
TwitterWithNeg=pd.concat(TwitterWithNeg, ignore_index=True)
TwitterWithout=pd.concat(TwitterWithout, ignore_index=True)

frames = [TwitterWithPos, TwitterWithNeg,TwitterWithNeu]
twitterAllWith = pd.concat(frames)
twitterAllWith = twitterAllWith.sample(frac=1).reset_index(drop=True) #Shuffle DF
"""  
TwitterWith=twitter
for i in range(0,len(twitter)):
    if pd.isnull(twitter._source[i].get('sentiment')):
        TwitterWith.drop(TwitterWith.index[[i]])
        
TwitterWith = TwitterWith.sample(frac=1).reset_index(drop=True)

twitter['language'] = pd.Series(np.zeros(len(twitter)));

for i in range(0,len(twitterAllWith)): #TO CHANGE !!! len(twitterAllWith)

    #ADD COLUMN TO ADD THE LANGUAGE
    sentence=twitterAllWith.iloc[i]["_source"].get('main')
    language='None';
    try:
        language=detect(sentence)
    except:
        e=1;
        
    twitterAllWith.ix[i,'language']=language

    if language=='en':
        words_filtered=filtering(stopwords_en,i)
        all_filtered_sentence_en.append((words_filtered,twitterAllWith._source[i].get('sentiment'))) 
    elif language=='fr':
        words_filtered=filtering(stopwords_fr,i)
        all_filtered_sentence_fr.append((words_filtered,twitterAllWith._source[i].get('sentiment')))
    elif language=='de':
        words_filtered=filtering(stopwords_de,i)
        all_filtered_sentence_de.append((words_filtered,twitterAllWith._source[i].get('sentiment')))
    elif language=='it':
        words_filtered=filtering(stopwords_it,i)
        all_filtered_sentence_it.append((words_filtered,twitterAllWith._source[i].get('sentiment')))                        


#CLASSIFYING
#word_features_en = get_word_features(get_words_in_tweets(all_filtered_sentence_en))
all_filtered_sentence_en_train=all_filtered_sentence_en[:round(0.8*len(all_filtered_sentence_en))]
#for i in range(0,len(all_filtered_sentence_en_train)):
#    training_set_en.append(extract_features(all_filtered_sentence_en_train[i],word_features_en))
training_set_en = nltk.classify.apply_features(extract_features, all_filtered_sentence_en_train)
classifier_en = nltk.NaiveBayesClassifier.train(training_set_en)

#word_features_fr = get_word_features(get_words_in_tweets(all_filtered_sentence_fr))
all_filtered_sentence_fr_train=all_filtered_sentence_fr[:round(0.8*len(all_filtered_sentence_fr))]
training_set_fr = nltk.classify.apply_features(extract_features, all_filtered_sentence_fr_train)
classifier_fr = nltk.NaiveBayesClassifier.train(training_set_fr)

#word_features_de = get_word_features(get_words_in_tweets(all_filtered_sentence_de))
all_filtered_sentence_de_train=all_filtered_sentence_de[:round(0.8*len(all_filtered_sentence_de))]
training_set_de = nltk.classify.apply_features(extract_features, all_filtered_sentence_de_train)
classifier_de = nltk.NaiveBayesClassifier.train(training_set_de)

#word_features_it = get_word_features(get_words_in_tweets(all_filtered_sentence_it))
all_filtered_sentence_it_train=all_filtered_sentence_it[:round(0.8*len(all_filtered_sentence_it))]
training_set_it = nltk.classify.apply_features(extract_features, all_filtered_sentence_it_train)
classifier_it = nltk.NaiveBayesClassifier.train(training_set_it)


#PRINT RESULTS
all_filtered_sentence_en_test=all_filtered_sentence_en[round(0.8*len(all_filtered_sentence_en)):]
testing_set_en = nltk.classify.apply_features(extract_features, all_filtered_sentence_en_test)
print(nltk.classify.accuracy(classifier_en, testing_set_en)*100)

all_filtered_sentence_fr_test=all_filtered_sentence_fr[round(0.8*len(all_filtered_sentence_fr)):]
testing_set_fr = nltk.classify.apply_features(extract_features, all_filtered_sentence_fr_test)
print(nltk.classify.accuracy(classifier_fr, testing_set_fr)*100)

all_filtered_sentence_de_test=all_filtered_sentence_de[round(0.8*len(all_filtered_sentence_de)):]
testing_set_de = nltk.classify.apply_features(extract_features, all_filtered_sentence_de_test)
print(nltk.classify.accuracy(classifier_de, testing_set_de)*100)

all_filtered_sentence_it_test=all_filtered_sentence_it[round(0.8*len(all_filtered_sentence_it)):]
testing_set_it = nltk.classify.apply_features(extract_features, all_filtered_sentence_it_test)
print(nltk.classify.accuracy(classifier_it, testing_set_it)*100)

####

print("--- %s seconds ---" % (time.time() - start_time))


In [10]:
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from langdetect import detect
from nltk.stem import PorterStemmer
import time
import string
start_time = time.time()

### CREATION STOP WORDS !!!   
with open('stopwords/de') as f:
    mylist = f.read().splitlines()
stopwords_de = []
for i in mylist:
    stopwords_de.append(i)
    
with open('stopwords/en') as f:
    mylist = f.read().splitlines()
stopwords_en = []
for i in mylist:
    stopwords_en.append(i)
    
with open('stopwords/it') as f:
    mylist = f.read().splitlines()
stopwords_it = []
for i in mylist:
    stopwords_it.append(i)
    
with open('stopwords/fr') as f:
    mylist = f.read().splitlines()
stopwords_fr = []
for i in mylist:
    stopwords_fr.append(i)
### FIN CREATION STOPWORDS
    
all_filtered_sentence_en=[]
all_filtered_sentence_de=[]
all_filtered_sentence_fr=[]
all_filtered_sentence_it=[]
    
def filtering(stop_words, index):
    sentence=twitterWith.iloc[index]["_source"].get('main');
    word_tokens = word_tokenize(sentence)
    words_filtered=[x.lower() for x in word_tokens if x not in stop_words if x not in string.punctuation if len(x)>=3 ]
    return words_filtered

def get_words_in_tweets(tweets):
    all_words = []
    for (words, sentiment) in tweets:
        all_words.extend(words)
    return all_words

def get_word_features(wordlist):
    wordlist = nltk.FreqDist(wordlist)
    word_features = wordlist.keys()
    return word_features

def extract_features(document):
    document_words = set(document)
    features = {}
    for word in get_word_features(document):
        features['contains(%s)' % word] = (word in document_words)
    return features

twitterWith=twitter
for i in range(0,len(twitter)):
    if pd.isnull(twitter._source[i].get('sentiment')):
        print(i)
        twitterWith.drop(twitterWith.index[[i]])
        
twitterWith = twitterWith.sample(frac=1).reset_index(drop=True)

twitter['language'] = pd.Series(np.zeros(len(twitter)));

for i in range(0,1000): #TO CHANGE !!! len(twitterWith)

    #ADD COLUMN TO ADD THE LANGUAGE
    sentence=twitterWith.iloc[i]["_source"].get('main')
    language='None';
    try:
        language=detect(sentence)
    except:
        e=1;
        
    twitterWith.ix[i,'language']=language

    if language=='en':
        words_filtered=filtering(stopwords_en,i)
        all_filtered_sentence_en.append((words_filtered,twitterWith._source[i].get('sentiment'))) 
    elif language=='fr':
        words_filtered=filtering(stopwords_fr,i)
        all_filtered_sentence_fr.append((words_filtered,twitterWith._source[i].get('sentiment')))
    elif language=='de':
        words_filtered=filtering(stopwords_de,i)
        all_filtered_sentence_de.append((words_filtered,twitterWith._source[i].get('sentiment')))
    elif language=='it':
        words_filtered=filtering(stopwords_it,i)
        all_filtered_sentence_it.append((words_filtered,twitterWith._source[i].get('sentiment')))                        


#CLASSIFYING
#word_features_en = get_word_features(get_words_in_tweets(all_filtered_sentence_en))
all_filtered_sentence_en_train=all_filtered_sentence_en[:round(0.8*len(all_filtered_sentence_en))]
#for i in range(0,len(all_filtered_sentence_en_train)):
#    training_set_en.append(extract_features(all_filtered_sentence_en_train[i],word_features_en))
training_set_en = nltk.classify.apply_features(extract_features, all_filtered_sentence_en_train)
classifier_en = nltk.NaiveBayesClassifier.train(training_set_en)

#word_features_fr = get_word_features(get_words_in_tweets(all_filtered_sentence_fr))
all_filtered_sentence_fr_train=all_filtered_sentence_fr[:round(0.8*len(all_filtered_sentence_fr))]
training_set_fr = nltk.classify.apply_features(extract_features, all_filtered_sentence_fr_train)
classifier_fr = nltk.NaiveBayesClassifier.train(training_set_fr)

#word_features_de = get_word_features(get_words_in_tweets(all_filtered_sentence_de))
all_filtered_sentence_de_train=all_filtered_sentence_de[:round(0.8*len(all_filtered_sentence_de))]
training_set_de = nltk.classify.apply_features(extract_features, all_filtered_sentence_de_train)
classifier_de = nltk.NaiveBayesClassifier.train(training_set_de)

#word_features_it = get_word_features(get_words_in_tweets(all_filtered_sentence_it))
all_filtered_sentence_it_train=all_filtered_sentence_it[:round(0.8*len(all_filtered_sentence_it))]
training_set_it = nltk.classify.apply_features(extract_features, all_filtered_sentence_it_train)
classifier_it = nltk.NaiveBayesClassifier.train(training_set_it)


#PRINT RESULTS
all_filtered_sentence_en_test=all_filtered_sentence_en[round(0.8*len(all_filtered_sentence_en)):]
testing_set_en = nltk.classify.apply_features(extract_features, all_filtered_sentence_en_test)
print(nltk.classify.accuracy(classifier_en, testing_set_en)*100)

all_filtered_sentence_fr_test=all_filtered_sentence_fr[round(0.8*len(all_filtered_sentence_fr)):]
testing_set_fr = nltk.classify.apply_features(extract_features, all_filtered_sentence_fr_test)
print(nltk.classify.accuracy(classifier_fr, testing_set_fr)*100)

all_filtered_sentence_de_test=all_filtered_sentence_de[round(0.8*len(all_filtered_sentence_de)):]
testing_set_de = nltk.classify.apply_features(extract_features, all_filtered_sentence_de_test)
print(nltk.classify.accuracy(classifier_de, testing_set_de)*100)

all_filtered_sentence_it_test=all_filtered_sentence_it[round(0.8*len(all_filtered_sentence_it)):]
testing_set_it = nltk.classify.apply_features(extract_features, all_filtered_sentence_it_test)
print(nltk.classify.accuracy(classifier_it, testing_set_it)*100)

####

print("--- %s seconds ---" % (time.time() - start_time))


9
18
39
40
47
50
59
75
80
83
84
87
90
106
116
118
126
132
167
176
224
226
239
249
280
291
308
320
326
329
331
332
343
351
366
375
383
404
414
417
444
445
455
463
480
484
486
496
534
538
543
549
550
551
575
596
597
630
649
661
678
692
694
700
711
712
714
726
740
785
794
811
814
821
829
850
888
892
906
909
911
923
933
950
980
1021
1096
1117
1133
1154
1173
1194
1241
1278
1344
1375
1473
1499
1511
1543
1696
1751
1758
1767
1777
1792
1843
1894
1906
1924
1954
1980
1991
2029
2067
2079
2083
2088
2095
2111
2119
2134
2137
2139
2143
2148
2150
2153
2154
2159
2167
2172
2177
2194
2207
2211
2212
2214
2244
2247
2253
2260
2281
2282
2290
2306
2309
2317
2369
2371
2375
2377
2381
2401
2425
2483
2510
2516
2557
2571
2734
2775
2811
2816
2941
2946
2958
3023
3027
3031
3035
3090
3117
3131
3150
3257
3345
3346
3421
3426
3446
3449
3557
3568
3573
3596
3611
3621
3695
3785
3809
3820
3825
3835
3856
3865
3904
3934
3987
4086
4105
4124
4125
4127
4200
4223
4228
4315
4435
4438
4443
4554
4563
4596
4640
4683
4701
4703
4745
4813

In [46]:
print(len(twitter))
print(len(twitterWith))

31058
31058


In [47]:
twitterWith=twitter
for i in range(0,len(twitter)):
    if pd.isnull(twitter._source[i].get('sentiment')):
        twitterWith.drop(twitterWith.index[[i]])

IndexError: index 29783 is out of bounds for axis 1 with size 29771

In [28]:
for i in range(0,400):
    print(twitterWith._source[i].get('sentiment'))

NEUTRAL
NEUTRAL
NEUTRAL
NEUTRAL
NEUTRAL
NEUTRAL
NEUTRAL
NEUTRAL
NEUTRAL
None
NEUTRAL
NEUTRAL
NEUTRAL
NEUTRAL
NEUTRAL
NEUTRAL
NEUTRAL
NEUTRAL
None
NEUTRAL
NEUTRAL
NEUTRAL
NEUTRAL
POSITIVE
NEUTRAL
NEUTRAL
NEUTRAL
NEUTRAL
NEUTRAL
NEUTRAL
NEUTRAL
NEUTRAL
NEUTRAL
NEUTRAL
NEUTRAL
NEUTRAL
NEUTRAL
NEUTRAL
NEUTRAL
None
None
POSITIVE
NEUTRAL
NEUTRAL
NEUTRAL
NEUTRAL
NEUTRAL
None
NEUTRAL
NEUTRAL
None
NEUTRAL
NEUTRAL
NEUTRAL
NEUTRAL
NEUTRAL
NEUTRAL
NEUTRAL
NEUTRAL
None
NEUTRAL
NEUTRAL
NEUTRAL
NEUTRAL
NEUTRAL
NEUTRAL
NEUTRAL
NEUTRAL
NEUTRAL
NEUTRAL
NEUTRAL
NEUTRAL
NEUTRAL
NEUTRAL
NEUTRAL
None
NEUTRAL
NEUTRAL
NEUTRAL
NEUTRAL
None
NEUTRAL
NEUTRAL
None
None
NEUTRAL
NEUTRAL
None
NEUTRAL
NEUTRAL
None
NEUTRAL
NEUTRAL
NEUTRAL
NEUTRAL
NEUTRAL
NEUTRAL
NEUTRAL
NEUTRAL
NEUTRAL
NEUTRAL
NEUTRAL
NEUTRAL
NEUTRAL
NEUTRAL
NEUTRAL
None
NEUTRAL
NEUTRAL
NEUTRAL
NEUTRAL
NEUTRAL
NEUTRAL
POSITIVE
NEUTRAL
NEUTRAL
None
NEUTRAL
None
NEUTRAL
NEUTRAL
NEUTRAL
NEUTRAL
NEUTRAL
NEUTRAL
NEUTRAL
None
NEUTRAL
NEUTRAL
NEUTRAL
NEUTRAL
